In [2]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="quick-starts-ws-122785")
exp = Experiment(workspace=ws, name="experiment-1")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-122785
Azure region: southcentralus
Subscription id: 82648f26-b738-43a4-9ebb-f954c9f1ff3a
Resource group: aml-quickstarts-122785


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
cpu_cluster_name = "exp1"
compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2'
                                                        ,vm_priority='dedicated'
                                                        ,min_nodes=0
                                                        ,max_nodes=4)

cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output=True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os
import train


# Specify parameter sampler
parameter_space={
                "C": [0.001,0.01,0.05,0.1,0.2,0.5,0.7],
                "max_iter": [10,20,40,70,100,130],
                }
ps = RandomParameterSampling(parameter_space, properties=None)

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1, slack_factor=0.3, slack_amount=None, delay_evaluation=0)### YOUR CODE HERE ###

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(entry_script="train.py",source_directory=".",compute_target=cpu_cluster)### YOUR CODE HERE ###

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(estimator=est,
                                hyperparameter_sampling=ps,
                                policy=policy,
                                primary_metric_name='validation_acc',
                                primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                max_total_runs=4,
                                max_concurrent_runs=4)### YOUR CODE HERE ###

In [9]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###

run = exp.submit(hyperdrive_config)
print(run.get_portal_url())
run.wait_for_completion()

https://ml.azure.com/experiments/experiment-1/runs/HD_4bbaf178-3797-426b-bc25-a76870cdd7ed?wsid=/subscriptions/82648f26-b738-43a4-9ebb-f954c9f1ff3a/resourcegroups/aml-quickstarts-122785/workspaces/quick-starts-ws-122785


{'runId': 'HD_4bbaf178-3797-426b-bc25-a76870cdd7ed',
 'target': 'exp1',
 'status': 'Canceled',
 'startTimeUtc': '2020-10-25T09:21:55.868601Z',
 'endTimeUtc': '2020-10-25T09:24:57.876589Z',
 'error': {'error': {'code': 'UserError',
   'message': 'User errors were found in at least one of the child runs.',
   'details': [],
   'debugInfo': {'message': '[(https://southcentralus.experiments.azureml.net/subscriptions/82648f26-b738-43a4-9ebb-f954c9f1ff3a/resourceGroups/aml-quickstarts-122785/providers/Microsoft.MachineLearningServices/workspaces/quick-starts-ws-122785/experiments/**SCRUBBED**/runs/HD_4bbaf178-3797-426b-bc25-a76870cdd7ed_0, {\'error\': {\n    "additional_properties": {\n        "componentName": null\n    },\n    "error": {\n        "additional_properties": {\n            "referenceCode": null,\n            "messageFormat": null,\n            "messageParameters": {},\n            "severity": null\n        },\n        "code": "UserError",\n        "message": "User program faile

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###
joblib.dump(run, 'model.joblib')


In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

### YOUR CODE HERE ###

ds = TabularDatasetFactory.from_delimited_files("https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv")

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(ds)### YOUR DATA OBJECT HERE ###

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=x,
    label_column_name=y,
    n_cross_validations=5)

In [ ]:
# Submit your automl run

### YOUR CODE HERE ###

from azureml.core.experiment import Experiment
experiment = Experiment(ws, "taxi-experiment")
local_run = experiment.submit(automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = local_run.get_output()
print(best_run)
print(fitted_model)